In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

# Define the path to the dump files folder
dump_folder = r"C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1"

# Function to process multiple dump files
def process_dump_files(dump_files):
    all_data = []
    for file in dump_files:
        with open(file, 'r') as f:
            lines = f.readlines()
        
        # Find the relevant section with atom data
        start_reading = False
        atom_data = []
        for line in lines:
            if "ITEM: ATOMS" in line:
                start_reading = True
                headers = line.strip().split()[2:]
                continue
            if start_reading:
                atom_data.append(line.strip().split())
        
        # Convert to DataFrame
        df = pd.DataFrame(atom_data, columns=headers)
        df = df.astype(float)
        
        # Add timestep column (extracted from filename or content)
        timestep = int(file.split("coord")[-1].split(".dump")[0])
        df["timestep"] = timestep
        
        all_data.append(df)
    
    return pd.concat(all_data, ignore_index=True)

# Get all dump files
dump_files = glob.glob(os.path.join(dump_folder, "coord*.dump"))
print(f"Found {len(dump_files)} dump files in {dump_folder}.")

# Process all dump files
df_atoms = process_dump_files(dump_files)

# Check if the external simulation parameters file exists
params_file = os.path.join(dump_folder, "slm_heat_transfer_data.csv")
if os.path.exists(params_file):
    data_params = pd.read_csv(params_file)
else:
    print(f"Warning: {params_file} not found. Creating a placeholder dataset.")
    data_params = pd.DataFrame({
        "laser_power": [100] * len(df_atoms),
        "scan_speed": [10] * len(df_atoms),
        "layer_thickness": [0.05] * len(df_atoms),
        "powder_density": [8000] * len(df_atoms),
    })

# Merge with extracted atom data
data = pd.concat([df_atoms, data_params], axis=1)

# Define input and output columns
input_features = ["laser_power", "scan_speed", "layer_thickness", "powder_density", "x", "y", "z", "timestep"]
target_feature = "f_Temp[0]"

# Extract inputs and outputs
X = data[input_features].values
y = data[target_feature].values.reshape(-1, 1)

# Normalize data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM Regressor
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'num_leaves': 31
}

print("Training LightGBM ...")
model = lgb.train(
    params,
    lgb_train,
    num_boost_round=100,
    valid_sets=[lgb_train, lgb_test],
    valid_names=["train","valid"],
)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save model
joblib.dump(model, os.path.join(dump_folder, "heat_transfer_lgbm_model.pkl"))
print("LightGBM Model trained on multiple timesteps and saved in the specified folder.")


Found 241 dump files in C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1.
Training LightGBM ...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002625 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1007
[LightGBM] [Info] Number of data points in the train set: 964000, number of used features: 4
[LightGBM] [Info] Start training from score 0.000795


C:\Users\MAY02\anaconda3\envs\SLM_Python\lib\site-packages\lightgbm\basic.py:374: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")


Mean Squared Error: 0.059695253633636924
LightGBM Model trained on multiple timesteps and saved in the specified folder.


In [5]:
# Example: New test input (modify these values as needed)
new_data = np.array([[100, 10, 0.05, 8000, 0.002, -0.013, 0.001, 1100000]])  # Replace with real values

# Apply the same scaling used in training
new_data_scaled = scaler_X.transform(new_data)  # Use the same scaler as training


# Predict temperature for the new sample
predicted_temperature = model.predict(new_data_scaled)

# Convert back to the original scale (if used)
predicted_temperature = scaler_y.inverse_transform(predicted_temperature.reshape(-1, 1))

print(f"Predicted Temperature: {predicted_temperature[0][0]}")


Predicted Temperature: 278.16456624750396


In [6]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

# Define the path to the dump files folder
dump_folder = r"C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1"

# Load trained model
model_path = os.path.join(dump_folder, "heat_transfer_lgbm_model.pkl")
model = joblib.load(model_path)
print("Model loaded successfully.")

# Function to process multiple dump files
def process_dump_files(dump_files):
    all_data = []
    for file in dump_files:
        with open(file, 'r') as f:
            lines = f.readlines()
        
        # Find the relevant section with atom data
        start_reading = False
        atom_data = []
        for line in lines:
            if "ITEM: ATOMS" in line:
                start_reading = True
                headers = line.strip().split()[2:]
                continue
            if start_reading:
                atom_data.append(line.strip().split())
        
        # Convert to DataFrame
        df = pd.DataFrame(atom_data, columns=headers)
        df = df.astype(float)
        
        # Add timestep column (extracted from filename or content)
        timestep = int(file.split("coord")[-1].split(".dump")[0])
        df["timestep"] = timestep
        
        all_data.append(df)
    
    return pd.concat(all_data, ignore_index=True)

# Get all dump files
dump_files = glob.glob(os.path.join(dump_folder, "coord*.dump"))
print(f"Found {len(dump_files)} dump files in {dump_folder}.")

# Process all dump files
df_atoms = process_dump_files(dump_files)

# Check if the external simulation parameters file exists
params_file = os.path.join(dump_folder, "slm_heat_transfer_data.csv")
if os.path.exists(params_file):
    data_params = pd.read_csv(params_file)
else:
    print(f"Warning: {params_file} not found. Creating a placeholder dataset.")
    data_params = pd.DataFrame({
        "laser_power": [100] * len(df_atoms),
        "scan_speed": [10] * len(df_atoms),
        "layer_thickness": [0.05] * len(df_atoms),
        "powder_density": [8000] * len(df_atoms),
    })

# Merge with extracted atom data
data = pd.concat([df_atoms, data_params], axis=1)

# Define input features for prediction
input_features = ["laser_power", "scan_speed", "layer_thickness", "powder_density", "x", "y", "z", "timestep"]
X_new = data[input_features].values[:5000]  # Select 5000 atoms for prediction

# Normalize using previously fitted scalers
scaler_X = StandardScaler()
X_new_scaled = scaler_X.fit_transform(X_new)

# Predict temperatures
y_pred = model.predict(X_new_scaled)

# Output predictions
predictions_df = data.iloc[:5000].copy()
predictions_df["Predicted_Temperature"] = y_pred

# Save predictions to a CSV file
predictions_file = os.path.join(dump_folder, "predicted_temperatures.csv")
predictions_df.to_csv(predictions_file, index=False)
print(f"Predicted temperatures saved to {predictions_file}")


Model loaded successfully.
Found 241 dump files in C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1.
Predicted temperatures saved to C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1\predicted_temperatures.csv
